In [125]:
#matching (K2/2MASS) + UCAC4(=Sampedro)
import catalog_match as cm
import os
import sys
import numpy as np


In [126]:
#x.loadcatalog('/home/ekaterina/Documents/Matching_Catalogs','_Sampedro_cluster_members_query.csv',(2,22),',', 1,[('UCAC4','U10'),('2MASSYN','U10')],debug=False)
dt=np.dtype(np.float64)
dtstr=np.dtype(np.unicode_, 10)
cat=np.genfromtxt('/home/ekaterina/Documents/Matching_Catalogs/cats/M67_Sampedro_cluster_members_query.csv' ,usecols=(2,22),delimiter='\t', skip_header=1,dtype=[('UCAC4','U10'),('2MASSYN',dt)])
samp_twomassyn=cat['2MASSYN'].tolist()
samp_twomassyn = [str(r) for r in samp_twomassyn]
samp_ucac4=cat['UCAC4'].tolist()
samp_ucac4 = [str(r) for r in samp_ucac4]
print(samp_ucac4[:10])
indices = [i for i, x in enumerate(samp_twomassyn) if x == -9999.99]
samp_twomassyn_with_ucac4=[i for j, i in enumerate(samp_twomassyn) if j not in indices]
ucac4_samp=[i for j, i in enumerate(samp_ucac4) if j not in indices]


['508-049159', '508-049161', '508-049164', '508-049170', '508-049171', '508-049173', '508-049174', '508-049178', '508-049180', '508-049181']


In [127]:
k2=np.genfromtxt('/home/ekaterina/Documents/Matching_Catalogs/cats/M67_k2_search.txt' ,usecols=(19,20),delimiter='\t', skip_header=2,dtype=[('UCAC4','U10'),('2MASS','U16')])

twomass_k2=k2['2MASS'].tolist()
twomass_k2 = [str(r) for r in twomass_k2]
ucac4_k2=k2['UCAC4'].tolist()
ucac4_k2 = [str(r) for r in ucac4_k2]
print(len(ucac4_k2))

indices = [i for i, x in enumerate(ucac4_k2) if x == '']
ucac4_k2=[i for j, i in enumerate(ucac4_k2) if j not in indices]
twomass_k2=[i for j, i in enumerate(twomass_k2) if j not in indices]
print(len(ucac4_k2))#habe ucac4 id
indices = [i for i, x in enumerate(twomass_k2) if x == '']
ucac4_k2=[i for j, i in enumerate(ucac4_k2) if j not in indices]
twomass_k2=[i for j, i in enumerate(twomass_k2) if j not in indices]
print(len(ucac4_k2))#habe ucac4 id und 2mass


752
358
358


In [128]:

before=len(ucac4_samp)
#ucac4_k2=[1,4,5,7,9,2,3]
#ucac4_samp=[6,5,4,8,11,1,0]
match=[]
for item in ucac4_k2:
    if item in ucac4_samp:
        match.append(item)
        ucac4_samp.remove(item)
print(before-len(ucac4_samp))

158


In [48]:
import pandas as pd
import numpy as np
cluster = 'M44'
loc1 = '/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_Sampedro_EPIC.csv'.format(cluster)
loc2 = '/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_everest_search.txt'.format(cluster)
loc3 = '/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_Sampedro_EPIC_PanSTARRS.csv'.format(cluster)

df = pd.read_csv(loc1)
everest = pd.read_csv(loc2,skiprows=[1])
ps = pd.read_csv(loc3)
#print(everest['K2 ID'].count())
df = df.merge(everest, left_on='id', right_on='K2 ID', how='inner')
#print(df.id.count())
df = df.merge(ps, left_on='id', right_on='id', how='left')
#print(len(set(df.id) & set(ps.id)))
p = 0.8
df = df[(df.P_M2_join_x > p) & (df.P_M2_kinem_x > p) & (df.P_M3_x > p) & (df.P_M1_x > p)]
#print(df.id.count())
#print(len(set(df.id) & set(ps.id)))
#df = df[(df.P_M1 > p)]
#grouped = df.groupby('UCAC4')
#df = df[df.angDist.isin(grouped.angDist.min().values)] 

sel = df[['id','gmag.1_x','rmag.1_x','imag.1_x','zmag.1','ymag',
          'J Mag','H Mag','K Mag','RAJ2000_x','DEJ2000_x']]
sel = sel.rename(index=str, columns={'id':'EPIC','gmag.1_x':'gmag',
                                     'rmag.1_x':'rmag','imag.1_x':'imag',
                                     'zmag.1':'zmag','J Mag':'J',
                                     'H Mag':'H','K Mag':'K',
                                     'RAJ2000_x':'RAJ2000','DEJ2000_x':'DEJ2000'})
sel = sel.drop_duplicates(subset='EPIC')
print(len(set(sel.EPIC) & set(ps.id)))
sel.describe()

10


,EPIC,gmag,rmag,imag,zmag,ymag,J,H,K,RAJ2000,DEJ2000
count,1.200000e+02,91.000000,93.000000,93.000000,10.000000,9.000000,119.000000,115.000000,117.000000,120.000000,120.000000
mean,2.119628e+08,12.545978,11.689462,11.297097,12.543670,12.604578,10.190933,9.909035,9.745761,130.117883,19.671125
std,1.975663e+04,2.147864,1.846278,1.763105,1.641948,1.412753,1.820098,1.559892,1.555754,0.266505,0.302395
min,2.119313e+08,9.431000,8.830000,7.958000,10.100500,10.885900,5.240000,5.026000,4.683000,129.616000,19.194300
25%,2.119463e+08,10.621000,10.116000,9.946000,11.386000,11.411000,8.910500,8.809000,8.722000,129.927000,19.419100
50%,2.119602e+08,12.386000,11.525000,11.356000,11.899000,11.850000,10.371000,10.085000,9.928000,130.088500,19.628400
75%,2.119810e+08,14.246000,13.114000,12.725000,14.344575,14.076800,11.702000,11.079500,10.918000,130.319500,19.948225
max,2.119968e+08,16.965000,15.551000,14.308000,14.408600,14.111300,13.043000,12.386000,12.234000,130.685000,20.201700


In [43]:
sel.head()

,EPIC,gmag,rmag,imag,zmag,ymag,J,H,K,RAJ2000,DEJ2000
0,247134522,15.267,14.383,14.021,NaN,NaN,12.684,12.238,12.122,71.1425,18.9991
1,247133675,14.072,13.353,13.063,NaN,NaN,11.891,11.577,11.486,71.2450,18.9909
2,247131381,NaN,NaN,NaN,14.7060,14.4881,13.311,12.673,12.489,71.2468,18.9694
4,247140208,11.482,11.106,10.969,10.8520,10.8210,9.994,9.854,9.748,71.1308,19.0535
5,247152215,15.939,14.917,14.518,14.2434,14.0872,13.038,12.550,12.373,71.1330,19.1686


In [44]:

sel.to_csv('share/{0}/{0}_parameter.csv'.format(cluster))

In [38]:
import pandas as pd
cluster = ['Pleiades','NGC_1647','M44','M67']
for c in cluster:
    df = pd.read_csv('/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_Sampedro_EPIC.csv'.format(c),
                    usecols=['id'])
    with open('/home/ekaterina/Documents/appaloosa/stars_shortlist/{0}/{0}_newmatch.txt'.format(c),'w') as f:
        f.write("\n".join(df.id.astype('str').tolist()))